In [ ]:
# Set your Google directions API key in the environment
# before starting the notebook server.

import os
KEY = os.environ.get('GOOGLE_API_KEY')

In [ ]:
# Details of the API call

# Make sure that the lookup date is not in the past
LOOKUP_DATE = '2018-10-15'
ORIGIN = 'Budapest Kelenföld railway station, Hungary'
ORIGIN = 'Balatonhenye, Hungary'
LANGUAGE = 'hu'

In [ ]:
MAX_TRANSFERS = 2

In [ ]:
# These are language-specific settings. The vehicle
# type symbol is used to indicate whether one has to
# take the train or the bus. In Hungarian, train is
# "vonat", hence the Ⓥ in my code.

VEHICLE_TYPE_SYMBOL = {
    'HEAVY_RAIL': 'Ⓥ',
    'BUS': 'Ⓑ'
}

# These replacements are done at the very end of
# generating the timetable HTML, with no particular 
# order to them. These are used to turn long stop names
# into abbreviated versions, e.g. "vasúti megállóhely",
# "train stop" to "vmh." You could replace it with
# abbreviations in your own language.

REPLACE_TEXT = {
    'autóbusz-állomás': 'aut.áll.',
    'autóbusz-forduló': 'aut.ford.',
    'autóbusz-pályaudvar': 'aut.pu.',
    'autóbusz-váróterem': 'aut.vt.',
    'vasútállomás': 'vá.',
    'pályaudvar': 'pu.',
    'vasúti megállóhely': 'vmh.',
    'bejárati út': 'bej.út'
}

In [ ]:
import requests
import json

import dateutil.parser as dp
from datetime import datetime

In [ ]:
# We need to pass a unix timestamp to the API to specify
# what point in time to calculate routes for.

t = '{}T00:00:00.000Z'.format(LOOKUP_DATE)
parsed_t = dp.parse(t)
departure_unix_timestamp = parsed_t.strftime('%s')

In [ ]:
# And we will turn unix timestamps back to dates to check
# that we are still on the same day as the query was for.

def ts_to_date(ts):
    return datetime.fromtimestamp(int(ts)).isoformat()[:10]

In [ ]:
request_data = {
    'origin': ORIGIN,
    'destination': DESTINATION,
    'mode': 'transit',
    'language': LANGUAGE,
}

request_data['key'] = KEY

request_url = 'https://maps.googleapis.com/maps/api/directions/json'

In [ ]:
def get_transit_plan(departure_time):
    rd = request_data.copy()
    rd['departure_time'] = str(departure_time)
        
    r = requests.get(request_url, params=rd)
    timetable_data = json.loads(r.text)
    if timetable_data['status'] != 'OK':
        print(timetable_data['status'])
    
    # only one route will be returned (for given dep time)
    route = timetable_data['routes'][0] 
    # only one leg will be returned, as no intermediate stops are possible in transit
    leg = route['legs'][0] 
    
    # ignore walking directions between stops
    transit_steps = [x for x in leg['steps'] if x['travel_mode'] != 'WALKING'] 
    
    # initialize result container
    transit_results = []
    
    # there will be steps (i.e. different vehicles one takes)
    # process these to get the parts of the data we need
    for step in transit_steps:
        # get the fields from the API response that we will
        # actually use
        s = step['transit_details']

        step_data = {}

        step_data['departure_stop'] = s['departure_stop']['name']
        step_data['departure_location'] = str(s['departure_stop']['location']['lat']) + \
                                          ',' + str(s['departure_stop']['location']['lng'])
        step_data['departure_time'] = s['departure_time']['text']
        step_data['departure_time_epoch'] = s['departure_time']['value']
        step_data['arrival_stop'] = s['arrival_stop']['name']
        step_data['arrival_location'] = str(s['arrival_stop']['location']['lat']) + \
                                        ',' + str(s['arrival_stop']['location']['lng'])
        step_data['arrival_time'] = s['arrival_time']['text']
        step_data['arrival_time_epoch'] = s['arrival_time']['value']
        step_data['vehicle'] = s['line']['vehicle']['name']
        step_data['vehicle_type'] = s['line']['vehicle']['type']
        step_data['headsign'] = s['headsign']
        step_data['line_short_name'] = s['line'].get('short_name')
        step_data['line_name'] = s['line'].get('name')
 
        transit_results.append(step_data)
    
    # we will need current the departure time to look for
    # the next one after it
    this_departure_time = transit_steps[0]['transit_details']['departure_time']['value']
    
    return(this_departure_time, transit_results, transit_steps)

In [ ]:
this_departure_time =  departure_unix_timestamp

full_transit_results = []
while True:
    # the departure time we pass to the API should be one second
    # after the previous departure time to get the next option

    this_departure_time = str(int(this_departure_time) + 1)
    this_departure_time, transit_results, _ = get_transit_plan(this_departure_time)
    print('.', end=' ')
    if ts_to_date(this_departure_time) != LOOKUP_DATE:
        # break the loop if we are on the next day
        break
    full_transit_results.append(transit_results)

In [ ]:
# skip results if they have too many transfers
filtered_results = [x for x in full_transit_results if len(x) <= MAX_TRANSFERS + 1]

In [ ]:
# gather all locations that are mentioned so that we can
# query the Google Location API to find out which locality
# (city, village, etc.) they are in.

locations = list(set([y['arrival_location'] for x in filtered_results for y in x] + 
                     [y['departure_location'] for x in filtered_results for y in x]))

In [ ]:
reverse_geocode_request_url = 'https://maps.googleapis.com/maps/api/geocode/json'

In [ ]:
# actually query the locations

location_lookup = {}
for loc in locations:
    r = requests.get(reverse_geocode_request_url, params={'latlng': loc, 'key': KEY})
    print('.', end=' ')
    loc_data = json.loads(r.text)
    location_name = [x['long_name'] for x in loc_data['results'][0]['address_components'] 
                         if 'locality' in x['types']][0]
    location_lookup[loc] = location_name

In [ ]:
# add the localities to the results dictionary

for res in filtered_results:
    for step in res:
        step['departure_locality'] = location_lookup[step['departure_location']]
        step['arrival_locality'] = location_lookup[step['arrival_location']]

In [ ]:
# get the result with the most legs; we will use this in formatting
# the resulting table in HTML

most_legs = max([len(x) for x in filtered_results])

# Creating the actual HTML output

In [ ]:
from jinja2 import Template

In [ ]:
# use the generated data to fill in the HTML template written 
# using the jinja2 templating language.

with open('timetable_template.html') as f:
    template = Template(f.read())
    
rendered_timetable = template.render(results=filtered_results, vst=VEHICLE_TYPE_SYMBOL, most_legs=most_legs)

for orig, new in REPLACE_TEXT.items():
    rendered_timetable = rendered_timetable.replace(orig, new)

with open('timetable.html', 'w') as f:
    f.write(rendered_timetable)